### Log into ArcGIS Online

In [1]:
from arcgis.gis import GIS
from datetime import datetime

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

gis = GIS("home")

In [2]:
cm_id = "03209b91e33f4c588007319d18cd6e70"
cm_item = gis.content.get(cm_id)

users_table = cm_item.tables[0]
inventory_table = cm_item.tables[1]
dependencies_table = cm_item.tables[2]

### WTF is the user.lastLogin property not just a datetime object???
Oh well we'll deal with the int...

In [3]:
# Get days since last login from datetime object
def get_time_delta(last_login):

    # user property is a Unix timestamp in miliseconds; convert to datetime object
    login_datetime = datetime.fromtimestamp(last_login / 1000)

    # Get time delta in days since last login
    delta_days = (datetime.now() - login_datetime).days

    return delta_days

In [4]:
# Get total storage of user's content
def get_total_storage(user):

    total_bytes = 0

    for folder in user.folders:
        for item in user.items(folder=folder):
            total_bytes += item.size

    total_mb = round(total_bytes / (1024*1024))

    return total_mb

# Temporary call in this block while testing so I don't
# keep having to calc my storage for every test run
my_storage = get_total_storage(gis.users.me)

### Assemble users to be added to User table

In [5]:
# Get all users in the org
all_users = gis.users.search(query="*", max_users=10000)

# initialize empty dict to hold users info for AGOL table
users_list = []

# Loop through users; populate users dictionary
for user in all_users:

    # Initialize two dicts I need for each row to be added to users table
    update_dict = {}
    user_dict = {}

    # Add k/v pairs to the inner dict
    user_dict["username"] = user.username
    user_dict["user_email"] = user.email
    user_dict["last_login"] = user.lastLogin
    user_dict["days_since_last_login"] = get_time_delta(user.lastLogin)

    # REFORMAT THIS BEFORE PUBLICATION...this is just so I don't have to get my storage while testing
    user_dict["total_storage"] = my_storage # get_total_storage(user)
    user_dict["user_role"] = user.role.lstrip("org_").title()

    # Add inner dict to outer dict w/key "attributes"
    update_dict["attributes"] = user_dict

    # Append the user dict to the users list
    users_list.append(update_dict)

print(f"Number of users to add to table: {len(users_list)}")

Number of users to add to table: 1


### THIS IS A BIG BLOCK OF FAKE USER DATA...
...because I'm building this prototype in an org from a Personal Use License that has, by definition, ONE user. 😭
<br>If you're adapting this script for your own purposes, and you actually have users, you can _simply DELETE the entire cell_ below. POOF. 
<br>Script will continue to work as expected, just without a bunch of fake data.

In [6]:
# Fake users data for use in creating Dashboard
# Because a (Personal Use License) org with a single user makes for a lame bar chart

# Fake Unix timestamps (miliseconds)
ts = [
    1740205197408,
    1721304097308,
    1757318197908,
    1748305167908,
    1756218197908,
    1755218197908,
]

fake_users = [
    {
        "attributes": {
            "username": "Matt",
            "user_email": "matt@aldermaps.com",
            "last_login": ts[0],
            "days_since_last_login": get_time_delta(ts[0]),
            "total_storage": 643,
            "user_role": "Publisher"
        }
    },
    {
        "attributes": {
            "username": "Eric",
            "user_email": "eric@aldermaps.com",
            "last_login": ts[1],
            "days_since_last_login": get_time_delta(ts[1]),
            "total_storage": 29,
            "user_role": "User"
        }
    },
    {
        "attributes": {
            "username": "Alanda",
            "user_email": "alanda@aldermaps.com",
            "last_login": ts[2],
            "days_since_last_login": get_time_delta(ts[2]),
            "total_storage": 92,
            "user_role": "User"
        }
    },
    {
        "attributes": {
            "username": "Christine",
            "user_email": "christine@aldermaps.com",
            "last_login": ts[3],
            "days_since_last_login": get_time_delta(ts[3]),
            "total_storage": 8,
            "user_role": "User"
        }
    },
    {
        "attributes": {
            "username": "Khaled",
            "user_email": "khaled@aldermaps.com",
            "last_login": ts[4],
            "days_since_last_login": get_time_delta(ts[4]),
            "total_storage": 133,
            "user_role": "Publisher"
        }
    },
    {
        "attributes": {
            "username": "Xiaoyi",
            "user_email": "xiaoyi@aldermaps.com",
            "last_login": ts[5],
            "days_since_last_login": get_time_delta(ts[5]),
            "total_storage": 3,
            "user_role": "Publisher"
        }
    },
]

# Tack my list of fake user dicts onto my own dict
# To disable addition of fake users, simply comment out the line below
# Or delete this entire cell
users_list = users_list + fake_users

### Write users to user table

In [7]:
# This script will effectively overwrite the output of the last run, 
# so before writing any new records we simply truncate tables
users_table.manager.truncate()

# Add users to table
users_update = users_table.edit_features(adds=users_list)